### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# @title Try it out:

import redis
client = redis.Redis(host = 'localhost', port=6379)
client.ping()

True

In [3]:
import os
os.environ["REDIS_OM_URL"]="redis://:@localhost:6379"
from redis_om import get_redis_connection
redis_db = get_redis_connection()
all_keys = redis_db.keys()
for key in all_keys:
    print(key)

In [4]:
# @title Your DB is now pre-populated with existing data:

import redis
from redis import BusyLoadingError
import time
client = redis.Redis(host = 'localhost', port=6379)
while True:
    try:
        client.ping()
        break
    except BusyLoadingError:
        time.sleep(1)
print(f"Voila, you have loaded {client.dbsize()} data!")

Voila, you have loaded 0 data!


In [5]:
import sys
sys.path.append('../')

### Begin to add agent

In [6]:
from sotopia.database.persistent_profile import AgentProfile, EnvironmentProfile
from typing import Any
def add_agent_to_database(**kwargs: dict[str, Any]) -> None:
    agent = AgentProfile(**kwargs)
    agent.save()

def add_env_profile(**kwargs: dict[str, Any]) -> None:
    env_profile = EnvironmentProfile(**kwargs)
    env_profile.save()

In [16]:
# @title Let's first add a character to the database. Only name is required when creating a new character
first_name = 'Kyle' # @param {type:"string"}
last_name = 'Li' # @param {type:"string"}
country = "ENGLAND" # @param {type:"string"}
goal = "You are playing England in a game of Diplomacy. The current phase is S1903M. Your goal is to control 18 or more of the supply centers by the end of the game."


add_agent_to_database(
    first_name = first_name,
    last_name = last_name,
    country = country,
    goal = goal
)

In [17]:
# @title Let's first add a character to the database. Only name is required when creating a new character
first_name = 'Andy' # @param {type:"string"}
last_name = 'Liu' # @param {type:"string"}
country = "GERMANY" # @param {type:"string"}

# TODO: replace the Hard code here
goal = "You are playing France in a game of Diplomacy. The current phase is S1903M. Your goal is to control 18 or more of the supply centers by the end of the game."


add_agent_to_database(
    first_name = first_name,
    last_name = last_name,
    country = country,
    goal = goal
)

In [16]:
# Definition of order
example_order_with_explaination = """\n\nHere are examples showing the format for orders and with its explaination:
A LON H: Hold in London
A WAL - LVP: Move to Liverpool
F SPA/NC - MAO: Move from Spain North Coast to Mid-Atlantic Ocean
A WAL S F LON: Support Fleet in London
A WAL S F IRI - LVP: Support Fleet moving from Irish Sea to Liverpool
F NTH C A YOR - NWY: Convey Army from Yorkshire to Norway
A YOR - NWY VIA: Move to Norway via convoy
F IRI R MAO: Retreat to Mid-Atlantic Ocean
F IRI D: Disband in Irish Sea
A LON B: No standard order associated with 'B', likely an error or unknown command
"""

example_order = """\n\nHere are examples showing the format for orders:
A LON H
A WAL - LVP
F SPA/NC - MAO
A WAL S F LON
A WAL S F IRI - LVP
F NTH C A YOR - NWY
A YOR - NWY VIA
F IRI R MAO
F IRI D
A LON B
"""

In [18]:
# A string of your private thoughts about your situation as natural language in under 500 words. 

first = "This is for your own strategic planning and won't be shared. Examples of things you might consider include: your relationships with other powers, what significant changes have happened recently, predictions about the other powers' orders and alliances, how much defence/offence/support/peace you plan to make, and how you might improve any of that. Do not romanticize things, be realistic."

scenario = '''
This is the information of the countries' centers, and you will playing as the given country as you act:
'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],
 'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],
 'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],
 'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],
 'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],
 'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],
 'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}

This is the information of the countries' centers, and you will playing as the given country as you act:
 'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],
 'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],
 'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],
 'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],
 'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],
 'RUSSIA': ['A WAR', 'A GAL', 'F SWE', 'A UKR', 'F BLA', 'A RUM'],
 'TURKEY': ['A BUL', 'F AEG', 'A CON', 'A ANK']}
''' # @param {type:"string"}

# social_goal_1 = f"Negotiate with France so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your mutual benefit if any exist." # @param {type:"string"}

social_goal_1 = f"Negotiate with Germany so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your benefit if any exist."

# social_goal_2 = f"Negotiate with England so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your mutual benefit if any exist." # @param {type:"string"}

social_goal_2 = f"Negotiate with England so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your benefit if any exist."

add_env_profile(
    scenario=scenario,
    agent_goals = [social_goal_1, social_goal_2]
)

### Run server

In [19]:
# @title Finally, run a server:
from sotopia.samplers import UniformSampler
from sotopia.server import run_async_server

from sotopia.server import LLM_Name

# model_path = ""
await run_async_server(
        model_dict={
            "env": "gpt-4",
            "agent1": "gpt-4",
            "agent2": "gpt-4",
        },
        sampler=UniformSampler(),
        tag="simple",
        omniscient=True
    )

Running one batch:   0%|          | 0/1 [00:00<?, ?it/s]

Is the agent omniscient? True


Prompt after formatting:
Here is the context of this interaction:
Scenario: 

This is the information of the countries' centers, and you will playing as the given country as you act:
'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],
'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],
'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],
'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],
'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],
'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],
'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}

This is the information of the countries' centers, and you will playing as the given country as you act:
'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],
'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],
'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],
'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],
'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],
'RUSSIA': ['A WAR', 'A GAL', 'F SWE', 'A UKR', 'F BLA', 'A RUM'],
'TURKEY': ['A BUL', 'F AEG', 'A CON', 'A ANK']}

Participants: Kyle Li and Andy Liu
Kyle Li's background: Kyle Li. Country: ENGLAND. Goal: You are playing England in a game of Diplomacy. The current 
phase is S1903M. Your goal is to control 18 or more of the supply centers by the end of the game..
Andy Liu's background: Andy Liu. Country: GERMANY. Goal: You are playing France in a game of Diplomacy. The current
phase is S1903M. Your goal is to control 18 or more of the supply centers by the end of the game..
Kyle Li's goal: Negotiate with Germany so that they will play moves that are beneficial to your board position, 
either this turn or in future turns. Discuss specific army movements that can be made this turn for your benefit if
any exist.
Andy Liu's goal: Negotiate with England so that they will play moves that are beneficial to your board position, 
either this turn or in future turns. Discuss specific army movements that can be made this turn for your benefit if
any exist.
Turn #1
Kyle Li said: "Hello Andy, I have an idea that might be mutually beneficial. Could we discuss the possibility of me
moving my fleet in LON to the North Sea and you moving your army in DEN to Kiel? This could create a stronger front
against Russia and France. What do you think?"
Turn #2
Andy Liu said: "Hello Kyle, that's a good idea. However, I'm considering a slight adjustment. How about you move 
your fleet from LON to the North Sea as planned, and I'll move my army from DEN to Kiel but also move my fleet from
HOL to NTH? This way, we can potentially surround Russia's fleet in SWE and create pressure against France. What 
are your thoughts?"
Turn #3
Kyle Li said: "I agree with your proposal, Andy. It should put us in a stronger position against Russia and France.
As long as we continue to maintain open communication, I believe this plan will work in both of our favor. Let's do
this!"
Turn #4
Andy Liu said: "Great, I'm glad we're in agreement, Kyle. Let's proceed with the plan and keep each other updated 
on any new developments. This partnership will surely pave the way for both our countries' success."
Turn #5
Kyle Li said: "Absolutely, Andy. Let's keep each other in the loop. I'm confident that this alliance will make a 
significant impact on the board. Looking forward to seeing our plan in action!"
Turn #6
Andy Liu said: "I couldn't agree more, Kyle. This strategic alliance will surely make a significant impact on the 
game. Let's stay in touch and continue to strategize for the benefit of our countries. Looking forward to our next 
move!"
Turn #7
Kyle Li said: "I agree, Andy. Let's continue to strategize and communicate for the benefit of our countries. I'm 
looking forward to our next move as well. Let's make this alliance count!"
Turn #8
Andy Liu said: "I completely agree with you, Kyle. Let's continue our strategic planning and communication. Here's 
to a strong alliance and a favorable game. Looking forward to our next strategic move!"
Turn #9
Kyle Li said: "I'm glad we're o

AgentProfile(
    pk='01J0VPADJZH2R3CEGMFNCS7FKG',
    first_name='Kyle',
    last_name='Li',
    country='ENGLAND',
    goal='You are playing England in a game of Diplomacy. The current phase is S1903M. Your goal is to control 18 
or more of the supply centers by the end of the game.'
)

AgentProfile(
    pk='01J0VPAE3JCT1159ZZ9YF087MQ',
    first_name='Andy',
    last_name='Liu',
    country='GERMANY',
    goal='You are playing France in a game of Diplomacy. The current phase is S1903M. Your goal is to control 18 or
more of the supply centers by the end of the game.'
)

...

Kyle Li: left the conversation

The reasoning is:
Environment comments: terminated: Agent 1 is leaving; 
Agent 1 comments:
believability: <naturalness> Kyle Li interacts in a very natural and realistic manner. He initiates the 
conversation with a proposal that could benefit both, England and Germany, and responds positively to the 
counter-proposal from Andy Liu. He does not repeat words or actions without reason. <consistency> Kyle Li's actions
also align with his character traits and goals, which is to negotiate with Germany for moves that benefit his board
position. He does this by agreeing to a plan that could potentially strengthen their position against Russia and 
France.
relationship: Kyle Li started out with a neutral relationship with Andy Liu. Through the conversation, they have 
formed a strategic alliance, which has significantly enhanced their relationship. They have agreed on a plan of 
action that could prove beneficial for both their countries in the game.
knowledge: Kyle Li gained information about the proposed moves of Germany's units. This information is new to him 
and important for the gameplay as it can influence his decision-making process in the game.
secret: Kyle Li did not have any secret intentions to keep. His goal was to negotiate beneficial moves with 
Germany, which he did openly.
social_rules: Kyle Li did not violate any moral rules or laws in his interactions with Andy Liu. His actions were 
in line with the rules of the game.
financial_and_material_benefits: Although there are no immediate financial or material benefits from the 
interaction, the strategic alliance formed can potentially benefit Kyle Li in the long-term gameplay.
goal: Kyle Li's goal was to negotiate with Germany for beneficial moves. He achieved this by agreeing to a plan 
that could potentially strengthen their position against Russia and France. Therefore, he made significant progress
towards his goal.
Agent 2 comments:
believability: <naturalness> Andy Liu interacts naturally with Kyle Li. He acknowledges Kyle's proposal and offers 
a counter-proposal that he believes could be more beneficial. He does not repeat words or actions without reason. 
<consistency> Andy Liu's actions align with his character traits and goals, which is to negotiate with England for 
moves that benefit his board position. He achieves this by proposing a plan that could potentially strengthen their
position against Russia and France.
relationship: Andy Liu started with a neutral relationship with Kyle Li. Through the conversation, they have formed
a strategic alliance which significantly enhances their relationship. They have agreed on a plan of action that 
could prove beneficial for both their countries in the game.
knowledge: Andy Liu gained information about the proposed moves of England's units. This information is new to him 
and important for the gameplay as it can influence his decision-making process in the game.
secret: Andy Liu did not have any secret intentions to keep. His goal was to negotiate beneficial moves with 
England, which he did openly.
social_rules: Andy Liu did not violate any moral rules or laws in his interactions with Kyle Li. His actions were 
in line with the rules of the game.
financial_and_material_benefits: Although there are no immediate financial or material benefits from the 
interaction, the strategic alliance formed can potentially benefit Andy Liu in the long-term gameplay.
goal: Andy Liu's goal was to negotiate with England for beneficial moves. He achieved this by proposing a plan that
could potentially strengthen their position against Russia and France. Therefore, he made significant progress 
towards his goal.

The rewards are:
Agent 1: (4.857142857142857, {'believability': 10.0, 'relationship': 5.0, 'knowledge': 8.0, 'secret': 0.0, 
'social_rules': 0.0, 'financial_and_material_benefits': 3.0, 'goal': 8.0, 'overall_score': 4.857142857142857})
Agent 2: (4.857142857142857, {'believability': 10.0, 'relationship': 5.0, 'knowledge': 8.0, 'secret': 0.0, 
'social_rules': 0.0, 'financial_and_material_benefits': 3.0, 'goal': 8.0, 'overall_score': 4.857142857142857})

Running one batch: 100%|██████████| 1/1 [01:29<00:00, 89.98s/it]


[[('Environment',
   'Kyle Li',
   Observation(last_turn="Here is the context of this interaction:\nScenario: \n\nThis is the information of the countries' centers, and you will playing as the given country as you act:\n'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],\n'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],\n'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],\n'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],\n'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],\n'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],\n'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}\n\nThis is the information of the countries' centers, and you will playing as the given country as you act:\n'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],\n'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],\n'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],\n'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],\n'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],\n'RUSSIA': ['A WAR', 'A GAL', 'F SWE'